<a href="https://colab.research.google.com/github/BStricks/pii/blob/main/data_pii.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PII NER Machine Learning exercise:

Background; traditional NER for PII detection consisted of large lists of REGEX patterns to identify when sensitive data was being efiltrated from a company.

Novel approach; we are now able to pre-train context-aware lanbuage models to out-perform these traditional approaches.



In [ ]:
!pip install datasets peft evaluate

In [3]:
###imports
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup
from tqdm import tqdm
from IPython.display import display, HTML
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    LoraConfig,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
)
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
import evaluate


In [5]:
x = torch.rand(5, 3)
print(x)
torch.cuda.is_available()

tensor([[0.8681, 0.4782, 0.5920],
        [0.5161, 0.2873, 0.0741],
        [0.4738, 0.5977, 0.5525],
        [0.4636, 0.9095, 0.2113],
        [0.6707, 0.6279, 0.6823]])


False

### Dataset EDA

1. **Dataset Inspection**: I load the `pii-masking-300k` dataset and examine individual data points

In [6]:
#1 - inspect dataset
dataset = load_dataset("ai4privacy/pii-masking-300k")
train = dataset['train']
validation = dataset['validation']
df_train = train.to_pandas()
print(type(dataset))
print(dataset.shape)
display(df_train)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.9k [00:00<?, ?B/s]

1english_openpii_30k.jsonl:   0%|          | 0.00/103M [00:00<?, ?B/s]

dutch_openpii_28k.jsonl:   0%|          | 0.00/102M [00:00<?, ?B/s]

french_openpii_31k.jsonl:   0%|          | 0.00/114M [00:00<?, ?B/s]

...

spanish_openpii_8k.jsonl:   0%|          | 0.00/27.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/177677 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/47728 [00:00<?, ? examples/s]

<class 'datasets.dataset_dict.DatasetDict'>
{'train': (177677, 9), 'validation': (47728, 9)}


,source_text,target_text,privacy_mask,span_labels,mbert_text_tokens,mbert_bio_labels,id,language,set
0,Subject: Group Messaging for Admissions Proces...,Subject: Group Messaging for Admissions Proces...,"[{'value': 'wynqvrh053', 'start': 287, 'end': ...","[[440, 453, ""USERNAME""], [430, 437, ""TIME""], [...","[Sub, ##ject, :, Group, Mess, ##aging, for, Ad...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40767A,English,train
1,- Meeting at 2:33 PM\n- N23 - Meeting at 11:29...,- Meeting at [TIME]\n- [USERNAME] - Meeting at...,"[{'value': '2:33 PM', 'start': 13, 'end': 20, ...","[[74, 81, ""TIME""], [50, 60, ""USERNAME""], [40, ...","[-, Meeting, at, 2, :, 33, PM, -, N, ##23, -, ...","[O, O, O, B-TIME, I-TIME, I-TIME, I-TIME, O, O...",40767B,English,train
2,Subject: Admission Notification - Great Britai...,Subject: Admission Notification - Great Britai...,"[{'value': '5:24am', 'start': 263, 'end': 269,...","[[395, 407, ""SOCIALNUMBER""], [358, 375, ""EMAIL...","[Sub, ##ject, :, Ad, ##mission, Not, ##ificati...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40768A,English,train
3,Card: KB90324ER\n Country: GB\n Building: ...,Card: [IDCARD]\n Country: [COUNTRY]\n Buil...,"[{'value': 'KB90324ER', 'start': 6, 'end': 15,...","[[390, 393, ""STATE""], [368, 378, ""CITY""], [346...","[Card, :, KB, ##90, ##32, ##4, ##ER, \, n, Cou...","[O, O, B-IDCARD, I-IDCARD, I-IDCARD, I-IDCARD,...",40768B,English,train
4,"N, WA14 5RW\n Password: r]iD1#8\n\n...and so...","N, WA14 5RW\n Password: [PASS]\n\n...and so ...","[{'value': 'r]iD1#8', 'start': 26, 'end': 33, ...","[[336, 352, ""DATE""], [26, 33, ""PASS""]]","[N, ,, W, ##A, ##14, 5, ##R, ##W, \, n, Pass, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PASS...",40768C,English,train
...,...,...,...,...,...,...,...,...,...
177672,nos de ansiedad generalizada. Los participante...,nos de ansiedad generalizada. Los participante...,[],[],"[nos, de, ans, ##ied, ##ad, generali, ##zada, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40766B,Spanish,train
177673,en actividades artísticas experimentaron una m...,en actividades artísticas experimentaron una m...,"[{'value': '51schirin@aol.com', 'start': 265, ...","[[447, 453, ""COUNTRY""], [403, 412, ""PASSPORT""]...","[en, actividades, artística, ##s, experiment, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40766C,Spanish,train
177674,"""building_number"": ""699"",...","""building_number"": ""[BUIL...","[{'value': '699', 'start': 41, 'end': 44, 'lab...","[[474, 482, ""LASTNAME1""], [433, 441, ""TIME""], ...","["", building, _, number, "", :, "", 699, "", ,, ""...","[O, O, O, O, O, O, O, B-BUILDING, I-BUILDING, ...",40766D,Spanish,train
177675,"},\n {\n ""partic...","},\n {\n ""partic...","[{'value': '1936boncheva@tutanota.com', 'start...","[[453, 461, ""CITY""], [374, 417, ""STREET""], [32...","[}, ,, {, "", participant, _, id, "", :, "", 1936...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-E...",40766E,Spanish,train


In [7]:
#1 - manually check
print(df_train.iloc[1921]['source_text'][:400])
print("######################################")
print(df_train.iloc[1921]['target_text'][:400])

mprovement.

Key Initiatives:
1. Data-Driven Decision Making
Utilize falabretti57's expertise in data analysis to implement a structured framework for data-driven decision-making. This approach will enable us to identify areas for improvement and monitor the effectiveness of interventions.

2. Enhanced Academic Support
Implement targeted interventions based on 619 622 0722's social support researc
######################################
mprovement.

Key Initiatives:
1. Data-Driven Decision Making
Utilize [USERNAME]'s expertise in data analysis to implement a structured framework for data-driven decision-making. This approach will enable us to identify areas for improvement and monitor the effectiveness of interventions.

2. Enhanced Academic Support
Implement targeted interventions based on [SOCIALNUMBER]'s social support researc


### Dataset Preparation

1. **Dataset Loading**: load the `pii-masking-300k` dataset and split it into training, validation, and test sets.
2. **Labeling**: Each text span is labeled as 1 (contains PII) or 0 (does not contain PII) based on the presence of keywords such as 'SOCIALNUMBER' and 'TEL'.
3. **Column Cleaning**: Irrelevant columns are removed to focus on the necessary text and label columns.
4. **Tokenization**: The text data is tokenized using the GPT-2 tokenizer, ensuring compatibility with the model.

Some notes on tokenization; The GPT-2 tokenizer uses BPE (byte pair encoding), used by OpenAI for tokenization when pretraining the GPT model. It’s used by a lot of Transformer models, including GPT, GPT-2, RoBERTa, BART, and DeBERTa. You can think of these as character ngram vectors, which can be combined to produce word vectors for out-of-vocabulary words. Ngram-based vectors convey less meaning than word vectors, but they’re still more efficient than vectors based on individual characters. BPE is a natural development of this idea.

When to train a new tokenizer; If a language model is not available in the language you are interested in, or if your corpus is very different from the one your language model was trained on, you will most likely want to retrain the model from scratch using a tokenizer adapted to your data. That will require training a new tokenizer on your dataset. Training a tokenizer is not the same as training a model! Model training uses stochastic gradient descent to make the loss a little bit smaller for each batch. It’s randomized by nature (meaning you have to set some seeds to get the same results when doing the same training twice). Training a tokenizer is a statistical process that tries to identify which subwords are the best to pick for a given corpus, and the exact rules used to pick them depend on the tokenization algorithm. It’s deterministic, meaning you always get the same results when training with the same algorithm on the same corpus.

In [10]:
#1 Dataset loading
train = load_dataset("ai4privacy/pii-masking-300k", split='train[:77677]')
validate = load_dataset("ai4privacy/pii-masking-300k", split='train[-77677:]')
test = load_dataset("ai4privacy/pii-masking-300k", split="validation")


#2 Function to add a 'label' column to the dataset indicating if there is PII (Personal Identifiable Information)
def add_is_pii(example):
    pii_mask = ['SOCIALNUMBER', 'TEL']
    example["label"] = 1 if any(word in example['span_labels'] for word in pii_mask) else 0
    return example

#3 Attempt to load a saved dataset; if it doesn't exist, load and process the original dataset
try:
    datasets = load_from_disk("data/saved_datasetv2")
    columns_to_remove_after_tokenizer = ["text",]
    print("Loaded saved Dataset")
except:
    columns_to_remove = ['target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set']
    columns_renaming_dict = {'source_text':'text'}

    train = train.map(add_is_pii)
    train = train.remove_columns(columns_to_remove)
    train = train.rename_columns(columns_renaming_dict)

    validate = validate.map(add_is_pii)
    validate = validate.remove_columns(columns_to_remove)
    validate = validate.rename_columns(columns_renaming_dict)

    test = test.map(add_is_pii)
    test = test.remove_columns(columns_to_remove)
    test = test.rename_columns(columns_renaming_dict)

    datasets = DatasetDict(
        { 'train': train,
          'test': test,
          'validation': validate}
    )
    columns_to_remove_after_tokenizer = ["text",]
    datasets.save_to_disk("data/saved_datasetv2")
    print("Split and Saved Dataset")


Loaded saved Dataset


In [11]:
#4 tokenization settings
#model name is
model_name_or_path = "gpt2"
#padding side
"""
With decoder-only models padding should be done on the left. This is because the output is a continuation of the input prompt
-- there would be gaps in the output without left padding
"""
padding_side = "left"

#call tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    padding_side=padding_side)

# Set the pad token if not already set
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    outputs = tokenizer(examples["text"], truncation=True, max_length=None)
    return outputs

# Apply the tokenization function to the dataset
tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=columns_to_remove_after_tokenizer,
)

# Rename the 'label' column to 'labels' which is the expected name for labels by the models of the
# transformers library
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/77677 [00:00<?, ? examples/s]

Map:   0%|          | 0/47728 [00:00<?, ? examples/s]

Map:   0%|          | 0/77677 [00:00<?, ? examples/s]

## Create DataLoaders for training, testing, and validation

In [12]:
#batch size is
batch_size = 16


In [13]:
# Collation function to standardize batch sizes
def collate_fn(examples):
    return tokenizer.pad(
        examples,
        padding="longest",
        return_tensors="pt")

# Instantiate dataloaders.
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=collate_fn,
    batch_size=batch_size)

test_dataloader = DataLoader(
    tokenized_datasets["test"],
    shuffle=False,
    collate_fn=collate_fn,
    batch_size=batch_size)

val_dataloader = DataLoader(
    tokenized_datasets["validation"],
    shuffle=False,
    collate_fn=collate_fn,
    batch_size=batch_size)


## Experiments

Fine-tuning large pretrained models is often prohibitively costly due to their scale. Parameter-Efficient Fine-Tuning (PEFT) methods enable efficient adaptation of large pretrained models to various downstream applications by only fine-tuning a small number of (extra) model parameters instead of all the model's parameters.

r; Lora attention dimension (the “rank”).

alpha; The alpha parameter for Lora scaling.

dropout; The dropout probability for Lora layers.

In [14]:
peft_config = LoraConfig(
    task_type="SEQ_CLS",
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1)

# Load the pre-trained GPT-2 model for sequence classification
gpt2_model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path,
    return_dict=True)

# Apply PEFT configuration to the model
model = get_peft_model(gpt2_model, peft_config)
model.print_trainable_parameters()

# Set the pad token in the model configuration
model.config.pad_token_id = model.config.eos_token_id

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 296,448 || all params: 124,737,792 || trainable%: 0.2377


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
from evaluate import evaluator

task_evaluator = evaluator("text-classification")

results_val = task_evaluator.compute(
    model_or_pipeline=gpt2_model,
    tokenizer=tokenizer,
    data=datasets['validation'],
    input_column='text',
    label_column='label',
    metric="accuracy",
    label_mapping={"LABEL_0": 0.0, "LABEL_1": 1.0},
    strategy="simple",
    random_state=0
)

results_val

In [17]:
results_test = task_evaluator.compute(
    model_or_pipeline=gpt2_model,
    tokenizer=tokenizer,
    data=datasets['test'],
    input_column='text',
    label_column='label',
    metric="accuracy",
    label_mapping={"LABEL_0": 0.0, "LABEL_1": 1.0},
    strategy="simple",
    random_state=0
)

results_test

{'accuracy': 0.35029751927589675,
 'total_time_in_seconds': 541.7190121199999,
 'samples_per_second': 88.10471652678021,
 'latency_in_seconds': 0.011350130156721419}

## Save initial model and accuracy to use inside training loop

In [18]:
best_model_accuracy = results_test['accuracy']
best_model = gpt2_model

## Optimizer and Learning Rate Scheduler Setup

In this section, we initialize the optimizer and the learning rate scheduler for training the model. We use the AdamW optimizer, which is known for its efficiency in training deep learning models by adapting the learning rate for each parameter and incorporating weight decay to prevent overfitting.

The learning rate scheduler is configured to gradually increase the learning rate during the initial warmup phase (6% of the total training steps) and then decrease it linearly for the remainder of the training. This helps in stabilizing the training process and improving the model's performance.

- **Optimizer**: `AdamW` with a specified learning rate and model parameters.
- **Scheduler**: Linear schedule with a warmup period, ensuring smooth adjustments to the learning rate during training.


In [19]:
#task is
task = "mrpc"
#peft type is
peft_type = PeftType.LORA
#device is
device = "cuda"
#number of epochs is
num_epochs = 20
#learning rate is
lr = 3e-4
#evaluation metric is
metric = evaluate.load("accuracy")

In [20]:
optimizer = AdamW(
    params=model.parameters(),
    lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [21]:
model.to(device)
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(test_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)
    if eval_metric['accuracy'] > best_model_accuracy:
        print('updating best model')
        best_model_accuracy = eval_metric['accuracy']
        best_model = model

100%|██████████| 2983/2983 [03:11<00:00, 15.61it/s]


epoch 0: {'accuracy': 0.947787462286289}
updating best model


100%|██████████| 2983/2983 [03:10<00:00, 15.63it/s]


epoch 1: {'accuracy': 0.9674614482065035}
updating best model


100%|██████████| 2983/2983 [03:10<00:00, 15.64it/s]


...

epoch 4: {'accuracy': 0.9740822996982903}
updating best model


100%|██████████| 2983/2983 [03:10<00:00, 15.64it/s]


epoch 5: {'accuracy': 0.9760727455581629}
updating best model


 25%|██▌       | 1235/4855 [02:50<08:21,  7.22it/s]


KeyboardInterrupt: 